In [1]:
import numpy as np
from typing import Tuple

from ipywidgets import IntSlider, Layout, VBox, Output

import scwidgets
from scwidgets.code import CodeDemo, CodeInput, ParameterPanel
from scwidgets.check import (
    Check,
    CheckableWidget,
    CheckRegistry,
    ChecksLog,
    assert_numpy_allclose,
    assert_numpy_floating_sub_dtype,
    assert_shape,
    assert_type,
)

In [2]:
scwidgets.get_css_style()

## ParameterPanel

##### Passes arguments to ipywidgets.interactive
##### see tutorial to see full possiblities https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html

In [3]:
ParameterPanel(a=5, b=5., c=["orange", "apple"], d=True)

ParameterPanel(children=(IntSlider(value=5, description='a', max=15, min=-5), FloatSlider(value=5.0, descripti…

In [4]:
ParameterPanel(a=5, b=5., c=["orange", "apple"], d=IntSlider(0,1, layout=Layout(width="500px")))

ParameterPanel(children=(IntSlider(value=5, description='a', max=15, min=-5), FloatSlider(value=5.0, descripti…

## CheckRegistry

In [5]:
def multi_param_check(use_fingerprint=False, failing=False):
    if failing:
        outputs_references = [
            (
                [1.0],
                np.array([2.0]),
            )
        ]
    else:
        outputs_references = [
            (
                np.array([1.0, 2.0]),
                np.array([3.0]),
            )
        ]

    if use_fingerprint:
        def fingerprint(output_parameter1, output_parameter2):
            return np.sum(output_parameter1) + np.sum(output_parameter2)
        for i, output_references in enumerate(outputs_references):
            outputs_references[i] = (fingerprint(*output_references),)
    else:
        fingerprint = None

    return Check(
        function_to_check=None,
        asserts=[
            assert_type,
            assert_shape,
            assert_numpy_floating_sub_dtype,
            assert_numpy_allclose,
        ],
        inputs_parameters=[
            {
                "parameter1": 1.0,
                "parameter2": 2.0,
                "parameter3": 3.0,
            }
        ],
        outputs_references=outputs_references,
        fingerprint=fingerprint,
    )

checks = [
    multi_param_check(False, False),
    multi_param_check(True, False),
    multi_param_check(False, True),
    multi_param_check(True, True)
]

class MockCheckableWidget(VBox, CheckableWidget):
    def __init__(self, check_registry):
        self._results = []
        CheckableWidget.__init__(self, check_registry)
        self._output = Output()
        VBox.__init__(self, [self._output])

    def compute_output_to_check(self, parameter1, parameter2, parameter3):
        return np.array([parameter1, parameter2]), np.array([parameter3])

    def handle_checks_result(self, result):
        self._output.clear_output()
        with self._output:
            if isinstance(result, Exception):
                raise result
            elif isinstance(result, ChecksLog):
                if result.successful:
                    print("Checks successful")
                else:
                    print(result.message())
            else:
                print(result)

In [6]:
check_registry = CheckRegistry()
check_registry

CheckRegistry(children=(HBox(children=(Button(description='Set all references', style=ButtonStyle()), Button(d…

In [7]:
widget = MockCheckableWidget(check_registry)
widget.add_check(checks)
widget

MockCheckableWidget(children=(Output(),))

## CodeDemo

In [8]:
def some_func(parameter1: float, parameter2: float, parameter3: float) -> Tuple[np.ndarray, np.ndarray]:
    import numpy as np
    return np.array([parameter1, parameter2]), np.array([parameter3])


demo = CodeDemo(
    some_func,
    check_registry,
    parameters=dict(parameter1=5., parameter2=5., parameter3=1.)
)

demo.add_check(checks)
demo

CodeDemo(children=(UpdateCueBox(children=(CheckCueBox(children=(CodeInput(code_theme='north', function_body='i…